# Finesse measurements


First step is to do a full sweep over 20 mu

In [1]:
import numpy as np 
import matplotlib.pyplot as plt


# Loads nicard and scope
manager.startModule('logic','cavitylogic')

0

In [2]:
cavitylogic._load_full_sweep(filepath='',
                            filename='2017-08-15_144400_full_sweep_data_calibrationOfStage.dat')

In [3]:
cavitylogic.time.shape

(1247952,)

In [ ]:
labels = ['reflection', 'NIcard', 'SG_velocity', 'SG_posion']
for i in range(4):
    plt.plot(cavitylogic.time,cavitylogic.volts[i], label = labels[i])
#plt.legend('ON')
plt.xlabel('')
plt.show()